In [ ]:
# Import necessary libraries

# Images
import cv2
import numpy as np
from pathlib import Path
import mediapipe as mp
import matplotlib.pyplot as plt

# PDF's
import fitz

In [ ]:
# Define paths
input_dir = Path("photo/images")
pdf_dir = Path("pdfs")
output_dir_face = Path("face_detected_MediaPipe")
cropped_dir_face = Path("cropped_faces_MediaPipe")
pdf_cropped_dir = Path("pdf_cropped_faces_MediaPipe")

output_dir_face.mkdir(parents=True, exist_ok=True)
cropped_dir_face.mkdir(parents=True, exist_ok=True)
pdf_cropped_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
# Initialize MediaPipe Face Detection
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7)

In [ ]:
def detect_faces_mediapipe(img, img_name, cropped_dir, annotated_dir):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = face_detection.process(img_rgb)

    face_count = 0
    if results.detections:
        for detection in results.detections:
            bboxC = detection.location_data.relative_bounding_box
            h, w, _ = img.shape
            x1 = int(bboxC.xmin * w)
            y1 = int(bboxC.ymin * h)
            x2 = int((bboxC.xmin + bboxC.width) * w)
            y2 = int((bboxC.ymin + bboxC.height) * h)

            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(w, x2), min(h, y2)

            face_crop = img[y1:y2, x1:x2]
            face_count += 1
            face_filename = cropped_dir / f"{img_name}_face_{face_count}.jpg"
            cv2.imwrite(str(face_filename), face_crop)
            print(f"Saved: {face_filename.name}")

            # Draw rectangle on the face
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

    annotated_path = annotated_dir / f"{img_name}_annotated.jpg"
    cv2.imwrite(str(annotated_path), img)
    print(f"Annotated image saved: {annotated_path.name}")

In [ ]:
def pdf_to_images(pdf_path, dpi=300):
    pdf_document = fitz.open(str(pdf_path))
    img_paths = []
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        zoom = dpi / 72
        mat = fitz.Matrix(zoom, zoom)
        pix = page.get_pixmap(matrix=mat)
        img_array = np.frombuffer(pix.samples, dtype=np.uint8).reshape((pix.height, pix.width, 3))
        img_paths.append((img_array, page_num + 1))
    return img_paths

In [ ]:
def process_images(max_images=25):
    image_counter = 0
    for folder in input_dir.iterdir():
        if not folder.is_dir():
            continue
        for img_path in folder.glob("*.jpg"):
            if image_counter >= max_images:
                print(f"Reached the max limit of {max_images} images.")
                return
            img = cv2.imread(str(img_path))
            detect_faces_mediapipe(img, img_path.stem, cropped_dir_face, output_dir_face)
            image_counter += 1

In [ ]:
def process_pdfs():
    pdf_files = list(pdf_dir.glob("*.pdf"))
    for pdf_file in pdf_files:
        print(f"Processing PDF: {pdf_file.name}")
        try:
            pages = pdf_to_images(pdf_file)
            for img_array, page_num in pages:
                img_name = f"{pdf_file.stem}_page_{page_num}"
                detect_faces_mediapipe(img_array, img_name, pdf_cropped_dir, output_dir_face)
        except Exception as e:
            print(f"Error processing {pdf_file.name}: {str(e)}")

In [ ]:
process_images(max_images=25)

In [ ]:
process_pdfs()

---

# MediaPipe

In [1]:
# Import necessary libraries
import os
import time
from pathlib import Path
from typing import List, Tuple, Optional

# Images
import cv2
import numpy as np
from pathlib import Path
import mediapipe as mp
import matplotlib.pyplot as plt

# PDF's
import fitz

In [ ]:
# Face detection using MediaPipe
class FaceDetector:
    # Configuration
    def __init__(self):
        self.input_dir = Path("photo/images")
        self.output_dir= Path("face_detected_MediaPipe")
        self.cropped_dir = Path("cropped_faces_MediaPipe")
        self.no_face_detected: List[Path] = []

        # Create directories
        self.output_dir.mkdir(parents=True, exist_ok=True)
        self.cropped_dir.mkdir(parents=True, exist_ok=True)

        # Initialize MediaPipe
        self.detector = mp.solutions.face_detection.FaceDetection(
            model_selection = 0, min_detection_confidence = 0.7
        )

    # Detect faces using MediaPipe with timing metrics
    def detect_faces(self, img_path: Path, is_pdf: bool = False) -> Optional[int]:
        start_time = time.time()
        
        try:
            img = cv2.imread(str(img_path)) if isinstance(img_path, Path) else img_path
            if img is None:
                print(f"Error reading image: {img_path}")
                return None

            # Detection
            detect_start = time.time()
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            results = self.detector.process(img_rgb)
            detect_time = time.time() - detect_start

            face_count = 0
            if results.detections:
                for detection in results.detections:
                    face_count += 1
                    self._process_detection(img, img_path, detection, face_count, is_pdf)

            # Save annotated image if not PDF
            if not is_pdf and isinstance(img_path, Path):
                output_path = self.output_dir / img_path.name
                cv2.imwrite(str(output_path), img)

            elapsed = time.time() - start_time
            print(f"MediaPipe: {getattr(img_path, 'name', 'PDF_page')} | "
                  f"Faces: {face_count} | Detect: {detect_time:.3f}s | Total: {elapsed:.3f}s")
            
            return face_count

        except Exception as e:
            print(f"Error processing {getattr(img_path, 'name', 'PDF_page')}: {str(e)}")
            return None
        
    # Process and save a single face detection
    def _process_detection(self, img: np.ndarray, img_path, detection, idx: int, is_pdf: bool) -> None:
            bboxC = detection.location_data.relative_bounding_box
            h, w = img.shape[:2]
            x1 = int(bboxC.xmin * w)
            y1 = int(bboxC.ymin * h)
            x2 = int((bboxC.xmin + bboxC.width) * w)
            y2 = int((bboxC.ymin + bboxC.height) * h)

            # Ensure coordinates are within image bounds
            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(w, x2), min(h, y2)

            # Crop and save face
            face_crop = img[y1:y2, x1:x2]
            crop_name = f"{getattr(img_path, 'stem', 'PDF_page')}_face_{idx}.jpg"
            output_path = self.cropped_dir / crop_name
            cv2.imwrite(str(output_path), face_crop)

            # Draw rectangle if not PDF
            if not is_pdf:
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)


    # Display sample images where no faces were detected
    def show_no_face_samples(self, sample_size: int = 3) -> None:
        print(f"\nImages without faces detected: {len(self.no_face_detected)}")
        for img_path in self.no_face_detected[:sample_size]:
            try:
                img = cv2.cvtColor(cv2.imread(str(img_path)), cv2.COLOR_BGR2RGB)
                plt.figure(figsize=(8, 6))
                plt.imshow(img)
                plt.title(f"No face detected: {img_path.name}")
                plt.axis('off')
                plt.show()
            except Exception as e:
                print(f"Error displaying {img_path}: {e}")

In [3]:
# Class for processing PDF files to detect faces.
class PDFProcessor:
    def __init__(self, face_detector: FaceDetector):
        self.pdf_dir = Path('pdfs')
        self.output_img_dir = Path('pdf_images_MediaPipe')
        self.pdf_cropped_dir = Path('pdf_cropped_faces_MediaPipe')
        self.detector = face_detector
        self.detector.cropped_dir = self.pdf_cropped_dir

        self.output_img_dir.mkdir(parents=True, exist_ok=True)
        self.pdf_cropped_dir.mkdir(parents=True, exist_ok=True)

    # Process all PDF files and return total processing time
    def process_pdfs(self) -> float:
        pdf_files = list(self.pdf_dir.glob("*.pdf"))
        if not pdf_files:
            print("No PDF files found")
            return 0.0

        start_time = time.time()
        print("\nStarting PDF processing...")
        
        for pdf_file in pdf_files:
            self._process_pdf(pdf_file)
            
        return time.time() - start_time

    # Process a single PDF file
    def _process_pdf(self, pdf_file: Path) -> None:
        print(f"\nProcessing PDF: {pdf_file.name}")
        pdf_start = time.time()

        try:
            doc = fitz.open(str(pdf_file))
            for page_num in range(len(doc)):
                page_start = time.time()
                page = doc.load_page(page_num)
                pix = page.get_pixmap(matrix=fitz.Matrix(300/72, 300/72))
                img_array = np.frombuffer(pix.samples, dtype=np.uint8).reshape(
                    (pix.height, pix.width, 3))
                img = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)

                # Process with face detector
                temp_path = self.output_img_dir / f"temp_{pdf_file.stem}_p{page_num}.png"
                cv2.imwrite(str(temp_path), img)
                faces = self.detector.detect_faces(temp_path, is_pdf=True)
                temp_path.unlink(missing_ok=True)

                page_time = time.time() - page_start
                print(f"Page {page_num+1}: {faces or 0} faces, processed in {page_time:.2f}s")

        except Exception as e:
            print(f"Error processing {pdf_file.name}: {str(e)}")
        
        pdf_time = time.time() - pdf_start
        print(f"Finished {pdf_file.name} in {pdf_time:.2f} seconds")

In [4]:
# Main execution
if __name__ == "__main__":
    print("Initializing MediaPipe face detector...")
    init_start = time.perf_counter()
    face_detector = FaceDetector()
    init_time = time.perf_counter() - init_start
    print(f"Detector initialized in {init_time:.2f} seconds\n")
    
    # Process images with metrics
    print("\n" + "="*50)
    print("Starting image processing...")
    img_start = time.perf_counter()
    
    image_counter = 0
    total_faces = 0
    total_time = 0.0
    max_images = 25
    
    for folder in face_detector.input_dir.iterdir():
        if not folder.is_dir():
            continue
            
        for img_path in folder.glob("*.jpg"):
            if image_counter >= max_images:
                break
                
            start_time = time.time()
            faces = face_detector.detect_faces(img_path)
            elapsed = time.time() - start_time
            
            total_time += elapsed
            if faces is not None:
                total_faces += faces
            image_counter += 1
            
            avg_time = total_time / image_counter
            avg_faces = total_faces / image_counter if image_counter > 0 else 0
            print(f"Progress: {image_counter}/{max_images} | "
                  f"Avg: {avg_time:.3f}s/img | "
                  f"Avg faces: {avg_faces:.1f}")
    
    img_elapsed = time.perf_counter() - img_start
    
    print("\n" + "="*50)
    print("IMAGE PROCESSING SUMMARY")
    print(f"Total images: {image_counter}")
    print(f"Images with faces: {image_counter - len(face_detector.no_face_detected)}")
    print(f"Images without faces: {len(face_detector.no_face_detected)}")
    print(f"Total faces: {total_faces}")
    print(f"Total time: {img_elapsed:.2f}s")
    print(f"Avg time/img: {total_time/image_counter:.3f}s")
    print(f"Avg faces/img: {total_faces/image_counter:.1f}")
    print("="*50 + "\n")
    
    if face_detector.no_face_detected:
        print(f"Showing {min(3, len(face_detector.no_face_detected))} samples without faces...")
        face_detector.show_no_face_samples(sample_size=3)

    # Process PDFs
    print("\n" + "="*50)
    print("Starting PDF processing...")
    pdf_start = time.perf_counter()
    
    pdf_processor = PDFProcessor(face_detector)
    pdf_time = pdf_processor.process_pdfs()
    pdf_elapsed = time.perf_counter() - pdf_start
    
    print("\n" + "="*50)
    print("PDF PROCESSING SUMMARY")
    print(f"Total time: {pdf_elapsed:.2f}s")
    if pdf_time > 0:
        print(f"PDF-only time: {pdf_time:.2f}s")
    print("="*50 + "\n")
    
    print("\n" + "="*50)
    print("FINAL SUMMARY")
    print(f"Total execution time: {time.perf_counter() - init_start:.2f}s")
    print(f"Total images processed: {image_counter}")
    print(f"Total faces detected: {total_faces}")
    print(f"Avg time/image: {total_time/image_counter:.3f}s")
    print(f"Avg faces/image: {total_faces/image_counter:.1f}")
    print("=".center(50,"="))

Initializing MediaPipe face detector...


AttributeError: 'FaceDetector' object has no attribute 'output_dir'